In [9]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
loan_data=pd.DataFrame.from_csv("C:/Users/bibin/Downloads/lending-club-loan-data/loan - Copy.csv")

C:\Users\bibin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2821: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
print('The shape is {}'.format(loan_data.shape))
print('Memory : {} Mb'.format(int(loan_data.memory_usage(deep=False).sum() / 1000000)))

The shape is (887379, 73)
Memory : 525 Mb


In [10]:
check_null = loan_data.isnull().sum(axis=0).sort_values(ascending=False)/float(len(loan_data))
check_null[check_null>0.6]

dti_joint                      0.999426
annual_inc_joint               0.999424
verification_status_joint      0.999424
il_util                        0.979020
mths_since_rcnt_il             0.976549
inq_last_12m                   0.975916
total_cu_tl                    0.975916
open_il_6m                     0.975916
open_il_12m                    0.975916
open_il_24m                    0.975916
open_acc_6m                    0.975916
total_bal_il                   0.975916
max_bal_bc                     0.975916
inq_fi                         0.975916
open_rv_12m                    0.975916
all_util                       0.975916
open_rv_24m                    0.975916
desc                           0.857976
mths_since_last_record         0.845553
mths_since_last_major_derog    0.750160
dtype: float64

In [11]:
loan_data.drop(check_null[check_null>0.5].index, axis=1, inplace=True)
loan_data.dropna(axis=0, thresh=30, inplace=True)

In [13]:
delete_me = ['policy_code','pymnt_plan', 'url', 'member_id', 'application_type', 'acc_now_delinq','emp_title', 'zip_code','title']
loan_data.drop(delete_me , axis=1, inplace=True)

In [14]:
# strip months from 'term' and make it an int
loan_data['term'] = loan_data['term'].str.split(' ').str[1]


In [17]:
# extract numbers from emp_length and fill missing values with the median
loan_data['emp_length'] = loan_data['emp_length'].str.extract('(\d+)').astype(float)
loan_data['emp_length'] = loan_data['emp_length'].fillna(loan_data.emp_length.median())


C:\Users\bibin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  


In [18]:
col_dates = loan_data.dtypes[loan_data.dtypes == 'datetime64[ns]'].index
for d in col_dates:
    loan_data[d] = loan_data[d].dt.to_period('M')

loan_data['amt_difference'] = 'same'
loan_data.loc[(loan_data['funded_amnt'] - loan_data['funded_amnt_inv']) > 0,'amt_difference'] = 'low'

In [19]:
# Make categorical

loan_data['delinq_2yrs_cat'] = 'no'
loan_data.loc[loan_data['delinq_2yrs']> 0,'delinq_2yrs_cat'] = 'yes'

loan_data['inq_last_6mths_cat'] = 'no'
loan_data.loc[loan_data['inq_last_6mths']> 0,'inq_last_6mths_cat'] = 'yes'

loan_data['pub_rec_cat'] = 'no'
loan_data.loc[loan_data['pub_rec']> 0,'pub_rec_cat'] = 'yes'

In [20]:
# Create new metric
loan_data['acc_ratio'] = loan_data.open_acc / loan_data.total_acc

In [21]:
features = ['loan_amnt', 'amt_difference', 'term',
            'installment', 'grade','emp_length',
            'home_ownership', 'annual_inc','verification_status',
            'purpose', 'dti', 'delinq_2yrs_cat', 'inq_last_6mths_cat',
            'open_acc', 'pub_rec', 'pub_rec_cat', 'acc_ratio', 'initial_list_status',
            'loan_status'
           ]


In [22]:
X_clean = loan_data.loc[loan_data.loan_status != 'Current', features]
X_clean.head()

,loan_amnt,amt_difference,term,installment,grade,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,delinq_2yrs_cat,inq_last_6mths_cat,open_acc,pub_rec,pub_rec_cat,acc_ratio,initial_list_status,loan_status
id,,,,,,,,,,,,,,,,,,,
1077501,5000.0,low,36,162.87,B,10.0,RENT,24000.0,Verified,credit_card,27.65,no,yes,3.0,0.0,no,0.333333,f,Fully Paid
1077430,2500.0,same,60,59.83,C,1.0,RENT,30000.0,Source Verified,car,1.00,no,yes,3.0,0.0,no,0.750000,f,Charged Off
1077175,2400.0,same,36,84.33,C,10.0,RENT,12252.0,Not Verified,small_business,8.72,no,yes,2.0,0.0,no,0.200000,f,Fully Paid
1076863,10000.0,same,36,339.31,C,10.0,RENT,49200.0,Source Verified,other,20.00,no,yes,10.0,0.0,no,0.270270,f,Fully Paid
1075269,5000.0,same,36,156.46,A,3.0,RENT,36000.0,Source Verified,wedding,11.20,no,yes,9.0,0.0,no,0.750000,f,Fully Paid


In [24]:

mask = (X_clean.loan_status == 'Charged Off')
X_clean['target'] = 0
X_clean.loc[mask,'target'] = 1

In [25]:
cat_features = ['term','amt_difference', 'grade', 'home_ownership', 'verification_status', 'purpose', 'delinq_2yrs_cat', 'inq_last_6mths_cat', 'pub_rec_cat', 'initial_list_status']


In [26]:
# Drop any residual missing value (only 24)
X_clean.dropna(axis=0, how = 'any', inplace = True)

X = pd.get_dummies(X_clean[X_clean.columns[:-2]], columns=cat_features).astype(float)
y = X_clean['target']



In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#from imblearn.over_sampling import SMOTE

X_scaled = preprocessing.scale(X)
print(X_scaled)
print('   ')
print(X_scaled.shape)

[[-1.06342493 -1.05092578  1.19311767 ..., -0.38225805  0.67046107
  -0.67046107]
 [-1.36792428 -1.46918883 -1.43223111 ..., -0.38225805  0.67046107
  -0.67046107]
 [-1.38010425 -1.3697377   1.19311767 ..., -0.38225805  0.67046107
  -0.67046107]
 ..., 
 [-0.91422025 -0.81752006 -1.14052569 ..., -0.38225805  0.67046107
  -0.67046107]
 [-1.18522467 -1.19819464  1.19311767 ...,  2.61603388  0.67046107
  -0.67046107]
 [-0.35089647 -0.09225746  0.02629599 ..., -0.38225805  0.67046107
  -0.67046107]]
   
(285571, 50)


In [28]:

def run_models(X_train, y_train, X_test, y_test, model_type='Non-balanced'):
    clfs = {'GradientBoosting': GradientBoostingClassifier(max_depth=6, n_estimators=100, max_features=0.3),
            'LogisticRegression': LogisticRegression(),
            # 'GaussianNB': GaussianNB(),
            'RandomForestClassifier': RandomForestClassifier(n_estimators=10)
            }
    cols = ['model', 'matthews_corrcoef', 'roc_auc_score', 'precision_score', 'recall_score', 'f1_score']

    models_report = pd.DataFrame(columns=cols)
    conf_matrix = dict()

    for clf, clf_name in zip(clfs.values(), clfs.keys()):
        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)
        y_score = clf.predict_proba(X_test)[:, 1]

        print('computing {} - {} '.format(clf_name, model_type))

        tmp = pd.Series({'model_type': model_type,
                         'model': clf_name,
                         'roc_auc_score': metrics.roc_auc_score(y_test, y_score),
                         'matthews_corrcoef': metrics.matthews_corrcoef(y_test, y_pred),
                         'precision_score': metrics.precision_score(y_test, y_pred),
                         'recall_score': metrics.recall_score(y_test, y_pred),
                         'f1_score': metrics.f1_score(y_test, y_pred)})

        models_report = models_report.append(tmp, ignore_index=True)
        conf_matrix[clf_name] = pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=False)
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_score, drop_intermediate=False, pos_label=1)

        plt.figure(1, figsize=(6, 6))
        plt.xlabel('false positive rate')
        plt.ylabel('true positive rate')
        plt.title('ROC curve - {}'.format(model_type))
        plt.plot(fpr, tpr, label=clf_name)
        plt.legend(loc=2, prop={'size': 11})
    plt.plot([0, 1], [0, 1], color='black')

    return models_report, conf_matrix

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y.values, test_size=0.4, random_state=0)
models_report, conf_matrix = run_models(X_train, y_train, X_test, y_test, model_type = 'Non-balanced')

print(models_report)

computing GradientBoosting - Non-balanced 
computing LogisticRegression - Non-balanced 
computing RandomForestClassifier - Non-balanced 
                    model  matthews_corrcoef  roc_auc_score  precision_score  \
0        GradientBoosting           0.037868       0.686362         0.467249   
1      LogisticRegression           0.020750       0.678488         0.483871   
2  RandomForestClassifier           0.058135       0.607008         0.313299   

   recall_score  f1_score    model_type  
0      0.005906  0.011665  Non-balanced  
1      0.001656  0.003301  Non-balanced  
2      0.036540  0.065447  Non-balanced  


In [39]:
conf_matrix['LogisticRegression']

Predicted,0,1
True,,
0,96080,32
1,18087,30


In [40]:
conf_matrix['RandomForestClassifier']

Predicted,0,1
True,,
0,94661,1451
1,17455,662


In [41]:
conf_matrix['GradientBoosting']

Predicted,0,1
True,,
0,95990,122
1,18010,107


In [36]:
from imblearn.over_sampling import SMOTE
index_split = int(len(X)/2)
X_train, y_train = SMOTE().fit_sample(X_scaled[0:index_split, :], y[0:index_split])
X_test, y_test = X_scaled[index_split:], y[index_split:]

#scores = cross_val_score(clf, X_scaled, y , cv=5, scoring='roc_auc')

models_report_bal, conf_matrix_bal = run_models(X_train, y_train, X_test, y_test, model_type = 'Balanced')

computing GradientBoosting - Balanced 
computing LogisticRegression - Balanced 
computing RandomForestClassifier - Balanced 


In [37]:
models_report_bal

,model,matthews_corrcoef,roc_auc_score,precision_score,recall_score,f1_score,model_type
0,GradientBoosting,0.117327,0.651606,0.265630,0.189596,0.221263,Balanced
1,LogisticRegression,0.165850,0.661715,0.201477,0.707869,0.313674,Balanced
2,RandomForestClassifier,0.084721,0.602852,0.217400,0.212565,0.214955,Balanced


In [38]:
conf_matrix_bal['LogisticRegression']

Predicted,0,1
True,,
0,64586,57651
1,6003,14546
